In [2]:
import pandas as pd

In [3]:
A = pd.read_csv('ATR.csv')
B = pd.read_csv('D_EW_B.csv')
C = pd.read_csv('D_EW_S.csv')
D = pd.read_csv('HHLL.csv')
E = pd.read_csv('LR_Explore.csv')
F = pd.read_csv('Pattern_Revv.csv')
G = pd.read_csv('SCTR_Trial.csv')
H = pd.read_csv('ZigZag.csv')

Data cleaning to rename each "Buy" field into df_Buy.
Applied on ATR, D_EW, LR_Explore, and ZigZag

In [7]:
# Rename column 'Buy' to 'ALR_Buy'
A.head()
A.rename(columns={'Buy': 'ALR_Buy'}, inplace=True)

In [17]:
B.head()
B.rename(columns={'Mega Buy by Larger Wave ': 'D_EW_Buy'}, inplace=True)

In [22]:
C.head()
C.rename(columns={'Mega Sellby Larger Wave': 'D_EW_Sell'}, inplace=True)

Index(['Ticker', 'Date/Time', 'Close', 'D_EW_Sell', 'SL'], dtype='object')

In [28]:
E.head()
E.rename(columns={'myBuy ': 'LR_Explore_Buy'}, inplace=True)

In [33]:
H.columns
H.rename(columns={'Buy': 'ZigZag_Buy'}, inplace=True)

In [36]:
# List of dataframes to merge
tables = [A, B, C, D, E, F, G, H]

# Start with an empty DataFrame for the final result
final_table = pd.DataFrame()

# Perform full outer join on Date/Time and create new rows for each Ticker
for table in tables:
    if 'Ticker' in table.columns:
        # Include 'Ticker' and 'Date/Time' in the melt operation
        temp = pd.melt(table, id_vars=['Date/Time', 'Ticker'], var_name='Field', value_name='Value')
    else:
        # If 'Ticker' is not present, use only 'Date/Time'
        temp = pd.melt(table, id_vars=['Date/Time'], var_name='Field', value_name='Value')

    final_table = pd.concat([final_table, temp], ignore_index=True)

# Pivot the final table to get back the desired structure
final_table = final_table.pivot_table(index=['Date/Time', 'Ticker'], columns='Field', values='Value', aggfunc='first').reset_index()

# Re-arrange columns to have 'Ticker' as the first column
cols = ['Ticker', 'Date/Time'] + [col for col in final_table.columns if col not in ['Ticker', 'Date/Time']]
final_table = final_table[cols]

# Display the final result
print(final_table)

# Optionally, save the result to a CSV file
final_table.to_csv('joined_tickers.csv', index=False)


Field          Ticker Date/Time ALR_Buy Alert BearBreak BullBreak Close  \
0          ATLANTA.NS  1/1/2024     NaN   0.0       NaN       NaN   NaN   
1             GATI.NS  1/1/2024     NaN   0.0       NaN       NaN   NaN   
2       TATACOFFEE.NS  1/1/2024     NaN   0.0       NaN       NaN   NaN   
3       TATAMETALI.NS  1/1/2024     NaN   0.0       NaN       NaN   NaN   
4         TINPLATE.NS  1/1/2024     NaN   0.0       NaN       NaN   NaN   
...               ...       ...     ...   ...       ...       ...   ...   
132118      ^CNXMETAL  9/9/2024     NaN   0.0       NaN       NaN   NaN   
132119    ^CNXPSUBANK  9/9/2024     NaN   0.0       NaN       NaN   NaN   
132120     ^CNXREALTY  9/9/2024     NaN   0.0       NaN       NaN   NaN   
132121       ^NSEBANK  9/9/2024     NaN   0.0       NaN       NaN   NaN   
132122          ^NSEI  9/9/2024     NaN   0.0       NaN       NaN   NaN   

Field  D_EW_B_Buy D_EW_Sell DnBars  ...    Rank    S1   SCTR   SL UpBars  \
0             NaN      